$$
\frac{dy}{dt} = -ky
$$
where $y(0)=y_0 \sim N(0,1)$, $k\sim N(0,1)$ independent

$$
y(t,k,y_0) = \sum_{i=0}^{P(P+3)/2} y_i(t) Q_i(\xi_1, \xi_2)
$$
where $\xi_1, \xi_2\sim N(0,1)$ and they are independent

| 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- |
| $1$ |  |  |  |  |
| $H_1$ | $J_1$ |  |  |  |
| $H_2$ | $H_1J_1$ | $J_2$ |  |  |
| $H_3$ | $H_2J_1$ | $H_1J_2$ | $J_3$ |  |
| $H_4$ | $H_3J_1$ | $H_2J_2$ | $H_1J_3$ | $J_4$ |

| 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- |
| $Q_0$ |  |  |  |  |
| $Q_1$ | $Q_2$ |  |  |  |
| $Q_3$ | $Q_4$ | $Q_5$ |  |  |
| $Q_6$ | $Q_7$ | $Q_8$ | $Q_9$ |  |
| $Q_{10}$ | $Q_{11}$ | $Q_{12}$ | $Q_{13}$ | $Q_{14}$ |

$$
k = \sum_{i=0}^P k_i H_i(\xi_1) = \sum_{i=0}^P k_i Q_{i(i+1)/2}
$$

$$
\sum_{l=0}^{P(P+3)/2} \frac{d y_l(t)}{dt} Q_l(\xi_1, \xi_2) = - \left(\sum_{i=0}^P k_i Q_{i(i+1)/2}\right) \left(\sum_{j=0}^{P(P+3)/2} y_j(t) Q_j(\xi_1, \xi_2)\right)
$$

$$
\frac{dy_l(t)}{dt} = - \frac{1}{\langle Q_l^2\rangle} \sum_{i=0}^P\sum_{j=0}^{P(P+3)/2} k_i y_j \langle Q_{i(i+1)/2}Q_jQ_l\rangle
$$

In [1]:
import numpy as np
import timeit
import numpy.polynomial.hermite_e as H 
from math import factorial
from scipy.stats import norm
from scipy.integrate import odeint
from matplotlib import pyplot as plt

%matplotlib notebook

In [2]:
def Phi(n):
    #define H_n
    coeffs = [0]*(n+1)
    coeffs[n] = 1
    return coeffs

def inner2_herm(n):       ###return the denominator when computing $k_i$
    return factorial(n)

def product3_herm(i,j,l):
    #compute \Phi_i*\Phi_j*\Phi_l
    return lambda x: H.hermeval(x, H.hermemul(H.hermemul(Phi(i),Phi(j)),Phi(l))) 

def inner3_herm(P,i,j,l):
    #compute <\Phi_i\Phi_j\Phi_l>
    
    #Set up Gauss-Hermite quadrature, weighting function is exp^{-x^2}
    m=(P+1)**2
    x, w=H.hermegauss(m)        
    inner=sum([product3_herm(i,j,l)(x[idx]) * w[idx] for idx in range(m)])         
    
    return inner/np.sqrt(2*np.pi)   #because of the weight


In [3]:
P=4
ki_herm = [0,1]+[0]*(P-1)
Inner3_herm = np.zeros((P+1,P+1,P+1)) #store all inner3_herm values
Inner2_herm = np.zeros(P+1)

for i in range(P+1):
    for j in range(P+1):
        for l in range(P+1):
            Inner3_herm[i,j,l] = inner3_herm(P,i,j,l)

for i in range(P+1):
    Inner2_herm[i] = inner2_herm(i)
            

In [101]:
def index(i):
    if i == 0:
        return np.array([0, 0])
    elif i == 1:
        return np.array([1, 0])
    elif i ==2:
        return np.array([0, 1])
    else:
        for n in range(2,P+1):
            q=2
            if i // np.int((n+2)*(n-1)/2) >=1 and i // np.int((n+3)*n/2+1) ==0:
                q = n
                v = i % np.int((q+2)*(q-1)/2+1)
                w = np.int(q-v)
                break
        return np.array([w,v])

In [95]:
index(4)

hi
hi again


array([1, 1])

\begin{align*}
\langle Q_i Q_j Q_l \rangle &= \langle H_{index(i)[0]}J_{index(i)[1]}H_{index(j)[0]}J_{index(j)[1]}H_{index(l)[0]}J_{index(l)[1]} \rangle\\
& = \langle H_{index(i)[0]}H_{index(j)[0]}H_{index(l)[0]}\rangle \langle J_{index(i)[1]}J_{index(j)[1]}J_{index(l)[1]}\rangle\\
& = Inner3_{herm}[index(i)[0],index(j)[0],index(l)[0]] \times Inner3_{herm}[index(i)[1],index(j)[1],index(l)[1]]
\end{align*}

\begin{align*}
\langle Q_i^2 \rangle &= \langle H_{index(i)[0]}J_{index(i)[1]}H_{index(i)[0]}J_{index(i)[1]}\\
&= \langle H_{index(i)[0]}H_{index(i)[0]}\rangle \langle J_{index(i)[1]}J_{index(i)[1]}\rangle\\
& = Inner2_{herm}[index(i)[0]] \times Inner3_{herm}[index(i)[1]]
\end{align*}

In [103]:
P = 4
ki_herm = [0,1]+[0]*(P-1)
# when P=4, the largest index of Q is P(P+3)/2
z = np.int(P*(P+3)/2+1)
Inner3_q = np.zeros((z,z,z)) #store all inner3_q values
Inner2_q = np.zeros(z)

for i in range(z):
    for j in range(z):
        for l in range(z):
            a = index(i)[0]
            b = index(j)[0]
            c = index(l)[0]
            d = index(i)[1]
            e = index(j)[1]
            f = index(l)[1]
            Inner3_q[i,j,l] = Inner3_herm[a,b,c]*Inner3_herm[d,e,f]

for i in range(z):
    a = index(i)[0]
    b = index(i)[1]
    Inner2_q[i] = Inner2_herm[a]*Inner2_herm[b]

$$
\frac{dy_l(t)}{dt} = - \frac{1}{\langle Q_l^2\rangle} \sum_{i=0}^P\sum_{j=0}^{P(P+3)/2} k_i y_j \langle Q_{i(i+1)/2}Q_jQ_l\rangle
$$

In [104]:
def ode_system_q(y, t, P):   
    #P indicates the highest degree
    dydt = np.zeros(np.int(P*(P+3)/2+1)) 
    for l in range(len(dydt)):
        dydt[l] = -(sum(sum(Inner3_q[np.int(i*(i+1)/2),j,l]*ki_herm[i]*y[j] for j in range(np.int(P*(P+3)/2+1))) for i in range(P+1)))/Inner2_q[l]
    return dydt
            

In [105]:
y_init = [0.0, 0.0, 1.0]+[0.0]*np.int(P*(P+3)/2-2)

sol_q = odeint(ode_system_q, y_init, np.linspace(0,1,101), args=(P, )) 

**Analytically**
$$
y(t,k,y_0) = y_0 e^{-kt}
$$

\begin{align*}
\bar{y}(t) &= \int_Y\int_X y_0 e^{-kt}\ d f_1(k)\ d f_2(y_0)\\
&= \int_Y y_0\ d f_2(y_0) \int_X e^{-kt}\ d f_1(k)\\
&=0
\end{align*}
$Y$ is the support of pdf of $\xi_2$, $X$ is the support of pdf of $\xi_1$

**PC:**
$$
y(t,k,y_0) = \sum_{i=0}^{P(P+3)/2} y_i(t) Q_i(\xi_1, \xi_2)
$$

$$
\bar{y}(t) = y_0(t)
$$

In [106]:
sol_q[100,0]    #return y_0(t=1)

-3.979993464696795e-17